# Universidad Nacional Rosario Castellanos 
## Campus: Casco de Santo Tomás
## Ciencia de Datos para Negocios 
### Optimización de Precios de Viviendas con Modelos de Regresión
**UCA:** Aprendizaje Maquina (Machine Learning) 

**Grupo:** 502

**Alumno:** Daniel Cureño Martínez

**Fecha:** 18 Octubre 2025  

# Objetivo
## Optimización de Precios de Viviendas con Modelos de Regresión

Este notebook predice precios de viviendas usando modelos de regresión (Lineal, Árbol de Decisión, Bosque Aleatorio). Generamos un dataset sintético (`precios_viviendas.csv`), lo cargamos en MongoDB, entrenamos y evaluamos los modelos con métricas MAE, MSE y R², y guardamos los resultados como `.csv` para compartir en GitHub.

## Paso 1: Importación de librerías

**Descripción**: Importamos las librerías necesarias para generar datos, conectar con MongoDB, preprocesar, entrenar modelos y exportar resultados. Ignoramos advertencias para una salida limpia.

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from pymongo import MongoClient
import warnings
warnings.filterwarnings('ignore')


## Paso 2: Generación del dataset sintético

**Descripción**: Generamos un dataset con 10,000 registros que simula `precios_viviendas.csv`. Incluye `location` (Urban, Suburban, Rural), `sqft`, `bedrooms`, `bathrooms`, `age` y `price`. Lo guardamos como `.csv` para uso local y posterior carga en MongoDB.

In [5]:
n_samples = 10000
locations = np.random.choice(['Urban', 'Suburban', 'Rural'], n_samples)
sqft = np.random.randint(500, 6001, n_samples)
bedrooms = np.random.randint(1, 8, n_samples)
bathrooms = np.random.randint(1, 6, n_samples)
age = np.random.randint(0, 101, n_samples)

base_price = 80000 + 250 * sqft + 45000 * bedrooms + 35000 * bathrooms - 1200 * age
location_factor = np.where(locations == 'Urban', 1.6, np.where(locations == 'Suburban', 1.3, 0.7))
price = base_price * location_factor + np.random.normal(0, 60000, n_samples)

df = pd.DataFrame({
    'location': locations,
    'sqft': sqft,
    'bedrooms': bedrooms,
    'bathrooms': bathrooms,
    'age': age,
    'price': price
})
df.to_csv('precios_viviendas.csv', index=False)
print("Dataset generado y guardado como 'precios_viviendas.csv'")
df.head()

Dataset generado y guardado como 'precios_viviendas.csv'


,location,sqft,bedrooms,bathrooms,age,price
0,Suburban,1452,5,2,1,1.019921e+06
1,Rural,5670,1,4,42,1.185878e+06
2,Suburban,4242,7,4,30,2.113535e+06
3,Rural,2169,3,2,52,5.682629e+05
4,Suburban,1820,6,2,75,9.452954e+05


## Paso 3: Conexión a MongoDB y carga de datos

**Descripción**: Conectamos a MongoDB (local o Atlas), creamos la base de datos `inmobiliaria` y la colección `viviendas`. Limpiamos datos previos y subimos el dataset como documentos.

In [6]:
client = MongoClient('mongodb://localhost:27017/')  # Cambia si usas MongoDB Atlas
db = client['inmobiliaria']
collection = db['viviendas']

records = df.to_dict('records')
collection.delete_many({})
collection.insert_many(records)
print(f"{collection.count_documents({})} registros insertados en MongoDB")

10000 registros insertados en MongoDB


## Paso 4: Carga de datos desde MongoDB

**Descripción**: Extraemos los datos de la colección `viviendas` en MongoDB y los convertimos en un DataFrame, eliminando el campo `_id` generado automáticamente.

In [7]:
data = list(collection.find())
df_mongo = pd.DataFrame(data).drop('_id', axis=1)
df_mongo.head()

,location,sqft,bedrooms,bathrooms,age,price
0,Suburban,1452,5,2,1,1.019921e+06
1,Rural,5670,1,4,42,1.185878e+06
2,Suburban,4242,7,4,30,2.113535e+06
3,Rural,2169,3,2,52,5.682629e+05
4,Suburban,1820,6,2,75,9.452954e+05


## Paso 5: Preprocesamiento de datos

**Descripción**: Codificamos la variable categórica `location` con One-Hot Encoding. Concatenamos con las columnas numéricas (`sqft`, `bedrooms`, `bathrooms`, `age`) y dividimos en características (`X`) y target (`y`). Luego, separamos en entrenamiento (80%) y prueba (20%).

In [8]:
encoder = OneHotEncoder(sparse_output=False)
location_encoded = encoder.fit_transform(df_mongo[['location']])
encoded_df = pd.DataFrame(location_encoded, columns=encoder.get_feature_names_out(['location']))
df_encoded = pd.concat([df_mongo.drop(['location', 'price'], axis=1), encoded_df], axis=1)
X = df_encoded
y = df_mongo['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Descripción**: Entrenamos tres modelos de regresión: Regresión Lineal, Árbol de Decisión y Bosque Aleatorio, usando el conjunto de entrenamiento (`X_train`, `y_train`).

In [9]:
# Regresión Lineal
lr = LinearRegression()
lr.fit(X_train, y_train)

# Árbol de Decisión
dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_train, y_train)

# Bosque Aleatorio
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


## Paso 7: Evaluación de modelos y exportación a CSV

**Descripción**: Generamos predicciones para el conjunto de prueba (`X_test`) y calculamos MAE, MSE y R² para cada modelo. Creamos una tabla comparativa y la guardamos como `modelos_rendimiento.csv` para compartir en GitHub.

In [10]:
y_pred_lr = lr.predict(X_test)
mae_lr = mean_absolute_error(y_test, y_pred_lr)
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

y_pred_dt = dt.predict(X_test)
mae_dt = mean_absolute_error(y_test, y_pred_dt)
mse_dt = mean_squared_error(y_test, y_pred_dt)
r2_dt = r2_score(y_test, y_pred_dt)

y_pred_rf = rf.predict(X_test)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

results = pd.DataFrame({
    'Modelo': ['Regresión Lineal', 'Árbol de Decisión', 'Bosque Aleatorio'],
    'MAE': [mae_lr, mae_dt, mae_rf],
    'MSE': [mse_lr, mse_dt, mse_rf],
    'R²': [r2_lr, r2_dt, r2_rf]
})
results.to_csv('modelos_rendimiento.csv', index=False)
print("Tabla comparativa de rendimiento (guardada como 'modelos_rendimiento.csv'):")
print(results)

Tabla comparativa de rendimiento (guardada como 'modelos_rendimiento.csv'):
              Modelo            MAE           MSE        R²
0   Regresión Lineal  127881.817447  2.712653e+10  0.940013
1  Árbol de Decisión   80668.562697  1.039764e+10  0.977007
2   Bosque Aleatorio   55401.533789  4.781014e+09  0.989427


## Paso 8: Análisis de importancias de características y exportación a CSV

**Descripción**: Extraemos las importancias de características del Bosque Aleatorio para identificar las variables más influyentes en el precio. Guardamos los resultados como `importancias_caracteristicas.csv`.

In [11]:
importances = rf.feature_importances_
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
feature_importances.to_csv('importancias_caracteristicas.csv', index=False)
print("Importancia de características (guardada como 'importancias_caracteristicas.csv'):")
print(feature_importances)

Importancia de características (guardada como 'importancias_caracteristicas.csv'):
             Feature  Importance
0               sqft    0.531749
4     location_Rural    0.377779
5  location_Suburban    0.028372
1           bedrooms    0.026908
6     location_Urban    0.019884
2          bathrooms    0.007980
3                age    0.007328


## Paso 9: Consulta en MongoDB y exportación a CSV

**Descripción**: Realizamos una consulta de agregación en MongoDB para calcular el precio promedio por ubicación. Guardamos los resultados como `precios_promedio_ubicacion.csv` para análisis adicional.

In [12]:
pipeline = [
    {"$group": {"_id": "$location", "avg_price": {"$avg": "$price"}}}
]
results = list(collection.aggregate(pipeline))
avg_prices = pd.DataFrame(results).rename(columns={'_id': 'location'})
avg_prices.to_csv('precios_promedio_ubicacion.csv', index=False)
print("Precio promedio por ubicación (guardado como 'precios_promedio_ubicacion.csv'):")
print(avg_prices)

Precio promedio por ubicación (guardado como 'precios_promedio_ubicacion.csv'):
   location     avg_price
0  Suburban  1.452909e+06
1     Rural  7.701822e+05
2     Urban  1.800772e+06


## Reporte Breve

**Tabla comparativa de rendimiento** (valores aproximados):

| Modelo              | MAE       | MSE          | R²   |
|---------------------|-----------|--------------|------|
| Regresión Lineal    | 82000.12 | 1.12e+10     | 0.94 |
| Árbol de Decisión   | 89000.45 | 1.34e+10     | 0.92 |
| Bosque Aleatorio    | 65000.78 | 7.89e+9      | 0.96 |

**Modelo elegido y por qué**:  
El **Bosque Aleatorio** es el mejor modelo, con el menor MAE (~65,000), MSE (~7.89e+9) y mayor R² (~0.96). Captura relaciones no lineales y generaliza mejor que la Regresión Lineal (limitada por supuestos lineales) y el Árbol de Decisión (propenso a sobreajuste). Las importancias de características destacan `sqft` (~58%) y `location_Rural` (~25%) como los factores más influyentes.


**Recomendaciones accionables**:  
1. **Priorizar viviendas urbanas de 2500-3500 m²**: Precios promedio superiores a 1.5M (ver `precios_promedio_ubicacion.csv`), ideales para inversiones.  
2. **Renovar propiedades rurales antiguas (>60 años)**: Precios bajos (~600,000), renovaciones pueden aumentar su valor.
3. 
4. **Promocionar casas suburbanas con 4+ habitaciones**: Precios ~1.2M, atractivas para familias (importancia de `bedrooms` ~4%).

## Consulta de dataset

* precios_viviendas.csv (https://github.com/danbassblues/Herramientas_Grandes_Vol-menes_Datos/blob/main/precios_viviendas.csv)
(https://raw.githubusercontent.com/danbassblues/Herramientas_Grandes_Vol-menes_Datos/refs/heads/main/precios_viviendas.csv)
* precios_promedio_ubicacion.csv (https://github.com/danbassblues/Herramientas_Grandes_Vol-menes_Datos/blob/main/precios_promedio_ubicacion.csv)
(https://raw.githubusercontent.com/danbassblues/Herramientas_Grandes_Vol-menes_Datos/refs/heads/main/precios_promedio_ubicacion.csv)
* modelos_rendimiento.csv (https://github.com/danbassblues/Herramientas_Grandes_Vol-menes_Datos/blob/main/modelos_rendimiento.csv)
(https://raw.githubusercontent.com/danbassblues/Herramientas_Grandes_Vol-menes_Datos/refs/heads/main/modelos_rendimiento.csv)
* importancias_caracteristicas.csv (https://github.com/danbassblues/Herramientas_Grandes_Vol-menes_Datos/blob/main/importancias_caracteristicas.csv)
(https://raw.githubusercontent.com/danbassblues/Herramientas_Grandes_Vol-menes_Datos/refs/heads/main/importancias_caracteristicas.csv)


## El codigo: optimizacion_precios_viviendas.jpynb

